## Business Undestanding


### Bestimmung der (betriebswirtschaftlichen) Problemstellung: 
Hier wird an den Anwender die Anforderung gestellt, das Data Mining Projekt betriebswirtschaftlich auszurichten. Dabei werden die operationalen und betriebswirtschaftlichen Zielkriterien formuliert.
>Aufgabenstellung



Bei der vorliegenden Projektaufgabe gilt es, drei Prognosemodelle zu erstellen, um auf Basis der vorliegenden Daten zu den Brunnen bestimmen zu können, welche der gegebenen Brunnen funktionieren, welche nicht und welche repariert werden müssen. Dabei soll anhand der ML-Pipeline, das heißt dem CRISP-DM Modell vorgegangen werden und die Güte der einzelnen Modelle anhand der AUC auf den Testdaten qualifiziert werden. 

### Situationsbewertung: 
Durch die Situationsbewertung werden vorhandene Software- sowie Personalressourcen, die für das Data Mining-Projekt zur Verfügung stehen, bestimmt. Zudem sind mögliche Risiken, die während des Data Mining-Projektes auftreten können, aufzuführen.
>Bewertung

### Bestimmung analytischer Ziele:
Ausgehend von der zuvor bestimmten betriebswirtschaftlichen Problemdefinition (z.B. zielgerichtete Ansprache der Kunden) müssen dazu die erforderlichen Datenanalyseaufgaben (z.B. Kundensegmentierung, Scoring-Verfahren zur Kampagnenoptimierung etc.) ermittelt werden. Zudem müssen die Erfolgskriterien für das Data Mining-Projekt bestimmt werden (z.B. Steigerung der Responsequote von Kampagnen um 3% bei weniger Ressourceneinsatz, Ansprache des Kunden nach Verhaltensmustern etc.).
>Ziele


### Erstellung des Projektplans: 
Der Projektplan beschreibt die beabsichtigten Ziele des Data Mining-Projektes, dazu gehört:
Auflistung der einzelnen Schritte mit Zeitspanne
Beurteilung möglicher Risiken (Verzögerungen, Ursachen für ein Scheitern des Projektes etc.)
Prüfung der zur Verfügung stehenden Ressourcen (wie Mitarbeiter ; Hardware; Software z.B. Data Warehouse, Data Mining-Werkzeuge; Datenbestand)
>Plan

## Data Understanding


### Daten sammeln: 
Hier werden die benötigten Daten für die Analyse beschaffen und, wenn erforderlich, in bereits bestehende Datenmengen integriert. Der Analyst sollte Probleme, die bei der Datenbeschaffung auftreten, stets dokumentieren, um mögliche Diskrepanzen bei einem Folgeprojekt in der Zukunft zu vermeiden.
>Datensätze

In [ ]:
# Einlesen der vorliegenden Datensätze
import pandas as pd
train_data = pd.read_csv("pump_train.csv")
test_data = pd.read_csv("pump_test.csv")

### Daten beschreiben: 
In diesem Schritt gilt es, ein allgemeines Verständnis für die Daten zu erlangen. Zudem werden die Eigenschaften der Daten beschrieben, wie z.B. Quantität der Daten, Formateigenschaften, Anzahl der Einträge und Felder sowie Eigenschaften der Felder. Die entscheidende Frage die sich der Analyst stellen sollte ist, ob die vorliegenden Daten der Datenanalyse genügen um das Projekt erfolgreich abzuschließen.
>Beschreibung der Daten

In [ ]:
# Verwenden verschiedener Funktionen, um den Datensatz an sich besser zu beschreiben
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data.shape

### Untersuchung der Daten: 
Zur Untersuchung der Daten werden erste Analysen mit den Daten betrieben. Hierzu werden Reports erstellt, um die ersten Erkenntnisse und Hypothesen zu visualisieren.
>erste Erkenntnisse/ Hypothesen

### Bewertung der Daten: 
An diesem Punkt wird die Qualität des Datenbestandes bewertet. Es sollte festgestellt werden, ob die Datenmenge für die Analyse ausreichend und verwendbar ist. Besonders ist auf fehlende Attributwerte zu achten.
>Bewertung der Daten

## Data Preperation

### Auswahl der Daten:
Die Auswahl der Daten für das Data Mining hängt stark von den Zielen ab, die man für das Data Mining-Projekt definiert. Hier spielen die Datenqualität und die technischen Gegebenheiten eine große Rolle. Es wird eine Selektion der Daten vorgenommen, wie z.B. eine Auswahl aller Kunden, die einen Umsatz von mehr als 100 Euro im Monat generieren. Am Ende dieses Prozesses sollte sich deutlich zeigen, welche Datenmengen(-Sets) in die Analyse aufgenommen werden oder ausgeschlossen werden.
>Auswahl der Daten

### Bereinigung der Daten: 
Ohne eine Bereinigung der Daten ist ein erfolgreiches Data Mining-Projekt fraglich. Es gilt eine saubere Datenmenge auszuwählen oder die Datenmenge muss bereinigt sein, um das gewünschte Ergebnis in der Modellierung zu erreichen.
>Bereinigung

### Transformation und Integration der Daten: 
Um die Daten in eine brauchbare Darstellungsform zu bringen, transformiert man die Daten. Die Transformation kodiert Daten und verändert deren Granularität durch Aggregation oder Disaggregation. Wichtige Kennzahlen, die für eine Analyse zu erstellen sind, könnten z.B. Umsatz pro Kunde, Deckungsbeitrag pro Kunde oder Umsatzanteil in Produktgruppe pro Kunde etc. sein.
>Transformation

### Format Data: 
In einigen Fällen muss für die Modellierung eine einfache Anpassung des Datenformates erfolgen, z.B. Anpassung des Datentyps.
>Anpassung der Daten

## Modeling

### Auswahl der Modellierungstechnik: 
Hier gilt es eine Modellierungstechnik auswählen, mit der man das Modell erstellt
>Aufzählung der möglichen Modelle

### Testmodell erstellen: 
Nach Auswahl des Modells wird ein Testmodell erstellt, um Qualität und Genauigkeit des Modells zu überprüfen. In überwachten Verfahren, wie der Klassifikation, ist es üblich, die Fehlerraten als Qualitätsmaß zu nutzen.
>Durchführung aller Modelle

### Bewertung des Modells: 
Hier ist das Modell nach der im Vorfeld definierten Data Mining-Zielsetzung zu bewerten. Des Weiteren gilt es die Data Mining-Ergebnisse in Bezug auf die betriebswirtschaftliche Fragestellung zu bewerten.
>Bewertung der Testmodelle, Auswahl des besten für die einzelnen drei Modelle

## Evaluation

### Bewerten der Resultate: 
In diesem Schritt bewertet man, inwieweit das Modell die Projektziele erreicht. Wenn die Ziele nicht erreicht sind, ist aufzuführen, aus welchen Gründen.
>RUC,...Bewerten

### Bewertung des Prozesses: 
Das Data Mining-Projekt wird rückblickend bewertet. Es wird festgestellt, ob alle wichtigen Faktoren betrachtet wurden und inwieweit die Attribute für zukünftige Data Mining-Projekte zu nutzen sind.
>Analyse, Bewertung

### Nächste Schritte festlegen: 
In diesem Schritt entscheidet der Projektleiter, ob das Projekt beendet ist und eingeführt wird.
>Ausblick geben

## Deployment

Die Deployment-Phase bildet in der Regel die Endphase eines Data Mining-Projektes. Hier werden die gewonnenen Erkenntnisse so geordnet und präsentiert, sodass für den Auftraggeber die Möglichkeit besteht dieses Wissen zu nutzen. Dazu gehört eine eventuelle Implementierungsstrategie, die Überwachung der Gültigkeit der Modelle, ein zusammenfassender Bericht und eine Präsentation.
>Fazit

CRISP-DM Modell Grundlagen vgl. https://datasolut.com/crisp-dm-standard/amp/